# ACRO Demonstration

In [1]:
import os
import sys
import pandas as pd
import numpy as np

In [2]:
# uncomment this line if acro is not installed
# ie you are in development mode
# sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO, add_constant, utils

### Instantiate ACRO

In [4]:
acro = ACRO()

INFO:acro:config: {'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'check_missing_values': False}


### Load test data
The dataset used in this notebook is the nursery dataset from OpenML.  
- The dataset can be read directly from OpenML using the code commented in the next cell.
- In this version, it can be read directly from the local machine if it has been downloaded. 
- The code below reads the data from a folder called "data" which we assume is at the same level as the folder where you are working.
- The path might need to be changed if the data has been downloaded and stored elsewhere.
 - for example use:  
    path = os.path.join("data", "nursery.arff")  
    if the data is in a sub-folder of your work folder

In [5]:
from scipy.io.arff import loadarff

path = os.path.join("../data", "nursery.arff")
data = loadarff(path)
df = pd.DataFrame(data[0])
df = df.select_dtypes([object])
df = df.stack().str.decode("utf-8").unstack()
df.rename(columns={"class": "recommend"}, inplace=True)
df.head()

,parents,has_nurs,form,children,housing,finance,social,health,recommend
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


### Convert 'more than 3' children to random between 4 and 10
Change the children column from categorical to numeric in order to be able to test some of the ACRO functions that require a numeric feature

In [6]:
print(f" children column entries in raw file {df.children.unique()}")

 children column entries in raw file ['1' '2' '3' 'more']


In [7]:
df["children"].replace(to_replace={"more": "4"}, inplace=True)
df["children"] = pd.to_numeric(df["children"])

df["children"] = df.apply(
    lambda row: row["children"]
    if row["children"] in (1, 2, 3)
    else np.random.randint(4, 10),
    axis=1,
)

# Examples of producing tabular output
We rely on the industry-standard package **pandas** for tabulating data.  
In the next few examples we show:
- first, how a researcher would normally make a call in pandas, saving the results in a variable that they can view on screen (or save to file?)
- then how the call is identical in SACRO, except that:
  - "pd" is replaced by "acro"
  - the researcher immediately sees a copy of what the TRE output checker will see.
  

### Pandas crosstab
This is an example of crosstab using pandas.  
We first make the call, then the second line print the outputs to screen.

In [8]:
table = pd.crosstab(df.recommend, df.parents)
print(table)

parents     great_pret  pretentious  usual
recommend                                 
not_recom         1440         1440   1440
priority           858         1484   1924
recommend            0            0      2
spec_prior        2022         1264    758
very_recom           0          132    196


### ACRO crosstab
This is an example of crosstab using ACRO.  
The INFO lines show the researcher what will be reported to the output checkers.
Then the (suppressed as necessary)  table is shown via. the print command as before.

In [9]:
safe_table = acro.crosstab(df.recommend, df.parents)
print(safe_table)

INFO:acro:outcome_df:
parents      great_pret  pretentious        usual
recommend                                        
not_recom            ok           ok           ok
priority             ok           ok           ok
recommend   threshold;   threshold;   threshold; 
spec_prior           ok           ok           ok
very_recom  threshold;            ok           ok
INFO:acro:get_summary(): fail; threshold: 4 cells suppressed; 
INFO:acro:add_output(): output_0_2023-06-22-12252532


parents     great_pret  pretentious   usual
recommend                                  
not_recom       1440.0       1440.0  1440.0
priority         858.0       1484.0  1924.0
recommend          NaN          NaN     NaN
spec_prior      2022.0       1264.0   758.0
very_recom         NaN        132.0   196.0


### ACRO crosstab with aggregation function

In [10]:
safe_table = acro.crosstab(df.recommend, df.parents, values=df.children, aggfunc="mean")
print(safe_table)

INFO:acro:outcome_df:
parents                        great_pret                    pretentious  \
recommend                                                                  
not_recom                              ok                             ok   
priority                               ok                             ok   
recommend   threshold; p-ratio; nk-rule;   threshold; p-ratio; nk-rule;    
spec_prior                             ok                             ok   
very_recom  threshold; p-ratio; nk-rule;                              ok   

parents                             usual  
recommend                                  
not_recom                              ok  
priority                               ok  
recommend   threshold; p-ratio; nk-rule;   
spec_prior                             ok  
very_recom                             ok  
INFO:acro:get_summary(): fail; threshold: 4 cells suppressed; p-ratio: 4 cells suppressed; nk-rule: 4 cells suppressed; 
INFO:acro:add_ou

parents     great_pret  pretentious     usual
recommend                                    
not_recom     3.086806     3.118750  3.159722
priority      2.614219     3.018868  3.138254
recommend          NaN          NaN       NaN
spec_prior    3.370425     3.327532  3.389182
very_recom         NaN     2.128788  2.153061


### ACRO pivot_table
This is an example of pivot table using ACRO.  
- Some researchers may prefer this to using crosstab.   
- Again the call syntax is identical to the pandas "pd.pivot_table"
- in this case the output is non-disclosive

In [11]:
table = acro.pivot_table(
    df, index=["parents"], values=["children"], aggfunc=["mean", "std"]
)
print(table)

INFO:acro:outcome_df:
                mean      std
            children children
parents                      
great_pret        ok       ok
pretentious       ok       ok
usual             ok       ok
INFO:acro:get_summary(): pass
INFO:acro:add_output(): output_2_2023-06-22-12252542


                 mean       std
             children  children
parents                        
great_pret   3.125694  2.248137
pretentious  3.115278  2.229202
usual        3.143750  2.270630


# Regression examples using ACRO

Again there is an industry-standard package in python, this time called **statsmodels**.
- The examples below illustrate the use of the ACRO wrapper standard statsmodel functions
- Note that statsmodels can be called using an 'R-like' format (using an 'r' suffix on the command names)
- most statsmodels functiobns return a "results object" which has a "summary" function that produces printable/saveable outputs 

### Start by manipulating the nursery data to get two numeric variables
- The 'recommend' column is converted to an integer scale

In [12]:
df["recommend"].replace(
    to_replace={
        "not_recom": "0",
        "recommend": "1",
        "very_recom": "2",
        "priority": "3",
        "spec_prior": "4",
    },
    inplace=True,
)
df["recommend"] = pd.to_numeric(df["recommend"])

new_df = df[["recommend", "children"]]
new_df = new_df.dropna()

### ACRO OLS 
This is an example of ordinary least square regression using ACRO.  
- Above recommend column was converted form categorical to numeric. 
- Now we perform a the linear regression between recommend and children. 
- This version includes a constant (intercept)
- This is just to show how the regression is done using ACRO.  
- **No correlation is expected to be seen by using these variables**

In [13]:
y = new_df["recommend"]
x = new_df["children"]
x = add_constant(x)

results = acro.ols(y, x)
results.summary()

INFO:acro:ols() outcome: pass; dof=12958.0 >= 10
INFO:acro:add_output(): output_3_2023-06-22-12252552


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              recommend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     9.906
Date:                Thu, 22 Jun 2023   Prob (F-statistic):            0.00165
Time:                        12:25:25   Log-Likelihood:                -25123.
No. Observations:               12960   AIC:                         5.025e+04
Df Residuals:                   12958   BIC:                         5.026e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2218      0.025     87.815      0.000       2.172       2.271
children       0.0207      0.007      3.147      0.002       0.008       0.034
==============================================================================
Omnibus:                    76924.329   Durbin-Watson:                   2.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1742.575
Skew:                          -0.485   Prob(JB):                         0.00
Kurtosis:                       1.488   Cond. No.                         6.90
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### ACRO OLSR
This is an example of ordinary least squares regression using the 'R-like' statsmodels api, i.e. from a formula and dataframe using ACRO 

In [14]:
results = acro.olsr(formula="recommend ~ children", data=new_df)
print(results.summary())

INFO:acro:olsr() outcome: pass; dof=12958.0 >= 10
INFO:acro:add_output(): output_4_2023-06-22-12252564


                            OLS Regression Results                            
Dep. Variable:              recommend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     9.906
Date:                Thu, 22 Jun 2023   Prob (F-statistic):            0.00165
Time:                        12:25:25   Log-Likelihood:                -25123.
No. Observations:               12960   AIC:                         5.025e+04
Df Residuals:                   12958   BIC:                         5.026e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.2218      0.025     87.815      0.0

### ACRO Probit
This is an example of probit regression using ACRO  
We use a different combination of variables from the original dataset.

Again, we support the use of R-lijke formulas - because we suprort R 

In [15]:
new_df = df[["finance", "children"]]
new_df = new_df.dropna()

y = new_df["finance"].astype("category").cat.codes  # numeric
y.name = "finance"
x = new_df["children"]
x = add_constant(x)

results = acro.probit(y, x)
print(results.summary())

INFO:acro:probit() outcome: pass; dof=12958.0 >= 10
INFO:acro:add_output(): output_5_2023-06-22-12252570


Optimization terminated successfully.
         Current function value: 0.693141
         Iterations 3
                          Probit Regression Results                           
Dep. Variable:                finance   No. Observations:                12960
Model:                         Probit   Df Residuals:                    12958
Method:                           MLE   Df Model:                            1
Date:                Thu, 22 Jun 2023   Pseudo R-squ.:               9.539e-06
Time:                        12:25:25   Log-Likelihood:                -8983.1
converged:                       True   LL-Null:                       -8983.2
Covariance Type:            nonrobust   LLR p-value:                    0.6789
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0063      0.019     -0.336      0.737      -0.043       0.031
children       0.0020      0.

### ACRO Logit
This is an example of logistic regression using ACRO using the statmodels function

In [16]:
results = acro.logit(y, x)
results.summary()

INFO:acro:logit() outcome: pass; dof=12958.0 >= 10
INFO:acro:add_output(): output_6_2023-06-22-12252576


Optimization terminated successfully.
         Current function value: 0.693141
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                finance   No. Observations:                12960
Model:                          Logit   Df Residuals:                    12958
Method:                           MLE   Df Model:                            1
Date:                Thu, 22 Jun 2023   Pseudo R-squ.:               9.539e-06
Time:                        12:25:25   Log-Likelihood:                -8983.1
converged:                       True   LL-Null:                       -8983.2
Covariance Type:            nonrobust   LLR p-value:                    0.6789
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0101      0.030     -0.336      0.737      -0.069       0.049
children       0.0032      0.008      0.414      0.679      -0.012       0.019
==============================================================================
"""

# ACRO functionality to let users manage their outputs

### 1: List current ACRO outputs
This is an example of using the print_output function to list all the outputs created so far

In [17]:
acro.print_outputs()

output_0_2023-06-22-12252532:
command: safe_table = acro.crosstab(df.recommend, df.parents)
summary: fail; threshold: 4 cells suppressed; 
outcome: parents      great_pret  pretentious        usual
recommend                                        
not_recom            ok           ok           ok
priority             ok           ok           ok
recommend   threshold;   threshold;   threshold; 
spec_prior           ok           ok           ok
very_recom  threshold;            ok           ok
output: [parents     great_pret  pretentious   usual
recommend                                  
not_recom       1440.0       1440.0  1440.0
priority         858.0       1484.0  1924.0
recommend          NaN          NaN     NaN
spec_prior      2022.0       1264.0   758.0
very_recom         NaN        132.0   196.0]
timestamp: 2023-06-22-12252532
comments: 


output_1_2023-06-22-12252537:
command: safe_table = acro.crosstab(df.recommend, df.parents, values=df.children, aggfunc="mean")
summary: fai

### 2: Remove some ACRO outputs before finalising 
This is an example of deleting some of the ACRO outputs.  
The name of the output that needs to be removed should be passed to the function remove_output.  
- Currently, all outputs names contain timestamp; that is the time when the output was created.  
- The output name can be taken from the outputs listed by the print_outputs function, 
- or by listing the results and choosing the specific output that needs to be removed

In [18]:
acro.remove_output("output_0_2023-06-19-13454368")

/Users/j4-smith/miniforge3/envs/sprint1/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3009: UserWarning: unable to remove output_0_2023-06-19-13454368, key not found
  result = self._run_cell(


### 3: Rename ACRO outputs before finalising
This is an example of renaming the outputs to provide a more descriptive name.  
The timestamp associated with the output name will not get overwritten

In [19]:
acro.rename_output("output_2_2023-06-19-13461618", "pivot_table")

/Users/j4-smith/miniforge3/envs/sprint1/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3009: UserWarning: unable to rename output_2_2023-06-19-13461618, key not found
  result = self._run_cell(


### 4: Add a comment to output
This is an example to add a comment to outputs.  
It can be used to provide a description or to pass additional information to the output checkers.

In [20]:
firstitem = list(acro.results.keys())[0]
acro.add_comments(firstitem, "Please let me have this data.")
acro.add_comments(firstitem, "6 cells were supressed in this table")

INFO:acro:a comment was added to output_0_2023-06-22-12252532
INFO:acro:a comment was added to output_0_2023-06-22-12252532


### 5: Add an unsupported output to the list of outputs
This is an example to add an unsupported outputs (such as images) to the list of outputs

In [21]:
acro.custom_output(
    "XandY.jfif", "This output is an image showing the relationship between X and Y"
)

INFO:acro:add_output(): output_7_2023-06-22-12252590


## 6 (the big one) Finalise ACRO
This is an example of the function _finalise()_ which the users must call at the end of each session.  
- It takes each output and saves it to a CSV file.    
- It also saves the SDC analysis for each output to a json file or Excel file  
  (depending on the extension of the name of the file provided as an input to the function)

In [22]:
output = acro.finalise("test_results.json")

INFO:acro:Directory outputs/ already exists
INFO:acro:output written to: test_results.json


# Over to Ben